In [15]:
import pandas as pd
import csv
import geopandas as gp
df = pd.read_csv('permits_master_1.csv', dtype={'CN': str, 'RRFFDD': str, 'region': str, 'forest': str, 'district': str})

In [5]:
zipfile = "zip:///Users/cmcglade/Documents/S_USA.RangerDistrict.zip"
districts = gp.read_file(zipfile)
districts = districts.to_crs('EPSG:5070')

# Select for currently authorized permits / easements and format dates

In [16]:
#select for active permits
active = ['ISSUED']
active_permits =  df[df['STATUS'].str.contains('|'.join(active)) == True].copy() 
#format dates 
active_permits['ISSUE_DATE'] = pd.to_datetime(active_permits['ISSUE_DATE'])
active_permits['EXP_EXPIRATION_DATE'] = pd.to_datetime(active_permits['EXP_EXPIRATION_DATE'])

# Make separate dataframes for relevant variables
Blank Expiration Dates

Has an expiration date value, so the FS likely has discretion over it

Expired for at least three years (Before 09/03/2018)

Ditch Bill Act easements

Recently issued permits (Since 1/1/2012)

In [18]:
#NO EXPIRATION DATE
bool_series = pd.isnull(active_permits['EXP_EXPIRATION_DATE'])
activepermitsnodate = active_permits[bool_series]

#HAS EXPIRATION DATE, the ones the FS has power over
fs_discretion = active_permits[~active_permits['EXP_EXPIRATION_DATE'].isnull()]

#EXPIRED AT LEAST THREE YEARS
active_permits['EXP_EXPIRATION_DATE'] = pd.to_datetime(active_permits['EXP_EXPIRATION_DATE'])
expired = active_permits[active_permits['EXP_EXPIRATION_DATE'] < '09/03/2018']

#DITCHBILLS
#there are very few instances in which a Use1 is a ditchbill but the use 2 and use 3 is not 
ditchbill =  activepermitsnodate[activepermitsnodate['USE1_NAME'].str.contains('PL 99-545') == True] 

#RECENT
recentpermits = active_permits[active_permits['ISSUE_DATE'] >= '01/01/2012']

# Group these variables by Forest 

In [21]:
#total permits by forest
totalpermits = active_permits.groupby('FORESTNAME')['FORESTNAME'].count().to_frame(name = 'total_structures').reset_index()

#total expired permits by forest
total_ex_permits = expired.groupby('FORESTNAME')['FORESTNAME'].count().to_frame(name = 'expired').reset_index()

#total blank exp per forest 
total_no_date = activepermitsnodate.groupby('FORESTNAME')['FORESTNAME'].count().to_frame(name = 'blank_expiration').reset_index()

#total ditch bills per forest
ditchbills = ditchbill.groupby('FORESTNAME')['FORESTNAME'].count().to_frame(name = 'ditchbill').reset_index()

#total discretionary permits per forest
discretion = fs_discretion.groupby('FORESTNAME')['FORESTNAME'].count().to_frame(name = 'discretion').reset_index()

#total recents per forest
total_recent = recentpermits.groupby('FORESTNAME')['FORESTNAME'].count().to_frame(name = 'recents').reset_index()

# Join the Group Bys into one dataframe

In [22]:
#merge total permits and expired permits
join1 = totalpermits.merge(total_ex_permits, left_on='FORESTNAME', right_on="FORESTNAME",how="left")

#merge total permits, expired permits and no exp date permits
join2 = join1.merge(total_no_date, left_on='FORESTNAME', right_on="FORESTNAME",how="left")

#merge total permits, expired, no exp, and discretion permits
join3 = join2.merge(discretion, left_on='FORESTNAME', right_on="FORESTNAME",how="left")

#merge total permits, expired, no exp, discretion and ditchbills
df6 = join3.merge(ditchbills, left_on='FORESTNAME', right_on="FORESTNAME",how="left")

# Determine the percent of each variable out of all permits

In [23]:
#percent of all permits expired
df6['percent_expired'] = df6['expired']/df6['total_structures']

#percent of all permits that have blank exp
df6['percent_blank_exp'] = df6['blank_expiration']/df6['total_structures']

#percent of the blank exp that are ditchbills
df6['percent_of_blank_exp_that_are_ditchbills'] = df6['ditchbill']/df6['blank_expiration']

#percent of all permits that the FS has discretion over
df6['percent_discretion'] = df6['discretion']/df6['total_structures']

#percent of all permits that are (definitely) ditchbills 
df6['percent_of_total_that_are_ditchbills'] = df6['ditchbill']/df6['total_structures']

# Create a csv that has permits per forest

In [24]:
df6.to_csv('permitsbyforest.csv')

# Group by variables on ranger district

In [34]:
#total permits by district
totalpermits_by_district = active_permits.groupby('DISTRICTNA')['DISTRICTNA'].count().to_frame(name = 'total_structures').reset_index()

#total expired permits by district
total_ex_permits_by_district = expired.groupby('DISTRICTNA')['DISTRICTNA'].count().to_frame(name = 'expired').reset_index()

#total no date by district
total_no_date_by_district = activepermitsnodate.groupby('DISTRICTNA')['DISTRICTNA'].count().to_frame(name = 'blank_expiration').reset_index()

#total ditchbills by district
ditchbills_by_district = ditchbill.groupby('DISTRICTNA')['DISTRICTNA'].count().to_frame(name = 'ditchbill').reset_index()

#total discretion by district
discretion_by_district = fs_discretion.groupby('DISTRICTNA')['DISTRICTNA'].count().to_frame(name = 'discretion').reset_index()

#total recents per district
total_recent_by_district = recentpermits.groupby('DISTRICTNA')['DISTRICTNA'].count().to_frame(name = 'recents').reset_index()


# Join the District Group Bys into on frame

In [35]:
#join total permits with the expired permits
join1 = totalpermits_by_district.merge(total_ex_permits_by_district, left_on='DISTRICTNA', right_on="DISTRICTNA",how="left")

#join total permits, expired and no exp permits
join2 = join1.merge(total_no_date_by_district, left_on='DISTRICTNA', right_on="DISTRICTNA",how="left")

#join total permits, expired, no exp and ditch bills
join3 = join2.merge(ditchbills_by_district, left_on='DISTRICTNA', right_on="DISTRICTNA",how="left")

#join total permits, expired, no exp, ditch bills and discretion
join4 = join3.merge(discretion_by_district, left_on='DISTRICTNA', right_on="DISTRICTNA",how="left")

#join total permits, expired, no exp, ditch bills and recents
df6 = join4.merge(total_recent_by_district, left_on='DISTRICTNA', right_on="DISTRICTNA",how="left")

# Calculate the percentage of the variables by district

In [36]:
#percent of all permits that are expired
df6['percent_expired'] = df6['expired']/df6['total_structures']
#percent of all permits that  have blank exp
df6['percent_blank_exp'] = df6['blank_expiration']/df6['total_structures']
#percent of blank exp that are ditchbills
df6['percent_of_blank_exp_that_are_ditchbills'] = df6['ditchbill']/df6['blank_expiration']
#percent of all permits that are ditchbills 
df6['percent_of_total_that_are_ditchbills'] = df6['ditchbill']/df6['total_structures']
#percent of all permits that the FS has discretion
df6['percent_discretion'] = df6['discretion']/df6['total_structures']

# Create a file that is permits per district

In [37]:
df6.to_csv('permitsbydistrict.csv')